In [1]:
import os
import yaml
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool, tool
from langchain_core.tools import ToolException
from typing import List, Optional, Dict, Any
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

In [2]:
@tool
def fake_tool():
    """This is a fake tool"""
    return "I am a fake tool"

In [3]:
class MyAgent:
    def __init__(self, tools, custom_prompt: str = None):
        self.llm = AzureChatOpenAI(
            model="gpt-4o",
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        )
        self.tools = tools
        self.agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent = create_tool_calling_agent(
            llm=self.llm,
            tools=self._enhance_agent_tools(tools),
            prompt=ChatPromptTemplate.from_messages([
                ("system", custom_prompt if custom_prompt else "You are an experienced assistant. You are able to perform any task asked by the user through all the tools you have access to. Your goal is to complete the user's task by using the tools you have access to. This is a core instruction: ALWAYS ALWAYS ALWAYS USE A STRING AS ID"),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]),
        )
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, memory=self.agent_memory, verbose=True)

    def run(self, user_message: str):
        if not isinstance(input, str):
            raise ValueError("Input must be a string")
        result = self.executor.invoke({"input": user_message})
        print(result)
        print("\n==================================================")
        print("--> Agent Response:")
        print(result['output'])
        print("\n==================================================")

    def loop_run(self):
        print("Type 'exit' to exit the loop")
        while True:
            user_message = input("Enter a message: ")
            if user_message == "exit":
                break
            result = self.executor.invoke({"input": user_message})
            print(result)

    def _try_except_tool(self, func):
        try:
            return func
        except ToolException as e:
            print(f"Error: {e}")

    def _enhance_agent_tools(self, agent_tools: List):
        enhanced_agent_tools = [
            StructuredTool(
                name=tool.name,
                description=tool.description,
                args_schema=tool.args_schema,
                func=self._try_except_tool(tool._run),
                handle_tool_error=True,
                handle_validation_error=True
            ) for tool in agent_tools
        ]
        return enhanced_agent_tools

In [4]:
@tool
def fake_tool():
    """This is a fake tool"""
    return "I am a fake tool"

agent = MyAgent(tools=[fake_tool])

In [5]:
agent.loop_run()

Type 'exit' to exit the loop


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Hello! How can I assist you today?

> Finished chain.
{'input': 'Hey there', 'chat_history': [HumanMessage(content='Hey there'), AIMessage(content='Hello! How can I assist you today?')], 'output': 'Hello! How can I assist you today?'}


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


I can assist you with a variety of tasks, including but not limited to:

- Answering questions and providing information
- Helping you with research or finding information online
- Assisting with writing, editing, or proofreading
- Managing schedules and reminders
- Offering technical support or troubleshooting
- Helping with calculations or data analysis
- Providing recommendations for books, movies, restaurants, etc.

Let me know what you need, and I'll do my best to help!

> Finished chain.
{'input': 'How CAN you assist me today?', 'chat_history': [HumanMessage(content='Hey there'), AIMessage(content='Hello! How can I assist you today?'), HumanMessage(content='How CAN you assist me today?'), AIMessage(content="I can assist you with a variety of tasks, including but not limited to:\n\n- Answering questions and providing information\n- Helping you with research or finding information online\n- Assisting with writing, editing, or proofreading\n- Managing schedules and reminders\n- Offe